In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

from sklearn.metrics import recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 200
pd.options.display.float_format = '{:.4f}'.format

In [4]:
path = 'C:/Users/user/Documents/GitHub/'
load_path = path + 'pm_data/'  # 데이터 불러오는 경로
save_path = path + 'pm_data/prepro/'  # 전처리 이후 데이터 저장하는 경로
result_path = path + 'pm_results/'

In [5]:
df = pd.read_csv(load_path + '2017_2020_data_외부데이터통합ver.csv', encoding='EUC-KR')
cate = pd.read_csv(load_path + 'catego3.csv', encoding='EUC-KR')

In [6]:
df_cp = df.copy()
df_pmbycmoto=df_cp[df_cp['당사자종별가해자_대분류']!='원동기장치자전거']
df_pmbycmoto.reset_index(inplace=True, drop=True)

In [7]:
dt=df_pmbycmoto.drop([x for x in cate[cate['method']=='delete'].loc[:,'column']],axis=1,inplace=False)
dt.drop(['index'],axis=1,inplace=True)
dt.drop(['당사자종별가해자_대분류'],axis=1,inplace=True)
dt.reset_index(inplace=True, drop=True)

In [9]:
dummy = [x for x in cate[cate['method']=='dummy'].iloc[:,0]]
binary = [x for x in cate[cate['method']=='binary'].iloc[:,0]]
ordinal = [x for x in cate[cate['method']=='ordinal'].iloc[:,0]]
leave = [x for x in cate[cate['method']=='LeaveOneOut'].iloc[:,0]]
binning = [x for x in cate[cate['method']=='binning'].iloc[:,0]]
numeric = [x for x in cate[cate['method']=='numeric'].iloc[:,0]]

# 저장 함수

In [10]:
def save_df(data):
    global save_pre
    if save_pre.empty:
        save_pre=data
    else:
        save_pre=pd.concat([save_pre,data],axis=1)

In [11]:
save_pre=pd.DataFrame()

# 전처리 함수

In [12]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import OneHotEncoder

In [13]:
def dummy_encoding(var, atr, newcol):
    df_temp = pd.DataFrame()
    odn = pd.Categorical(var, categories = atr, ordered = False)
    odn_v = pd.DataFrame(odn.codes)
    enc = OneHotEncoder()
    enc_df = pd.DataFrame(enc.fit_transform(odn_v).toarray())

    enc_df.columns = newcol
    df_temp= enc_df.iloc[:,1:] # 앞에 변수 빼줌으로써, dummy 변환  
    return df_temp

In [14]:
def binary_encoding(data, atr_t,atr_f, newname): # 변수, 1로 변환할 변수명, 새로운 변수 이름
    bn = []  
    for index, row in data.iterrows():
        if row[0] in atr_t:
            bn.append(1)
        elif row[0] in atr_f:
            bn.append(0)
        else:
            bn.append(-1)

    bv = pd.DataFrame(bn)
    bv.columns = newname
    return bv

In [15]:
def make_ordinal(data, m_list, col,w):
    series=data.reset_index(drop=True).copy()
    
    for i in range(len(m_list)):
        num=0
        for index, row in series.iterrows():
            if row[0] in m_list[i]:
                series.loc[num]=w[i]
            else:
                series.loc[num]=0.5
            
            num=num+1
    series.columns = col
    return series

In [16]:
from sklearn.base import BaseEstimator, TransformerMixin
import category_encoders as ce

In [17]:
class TargetEncoder(BaseEstimator, TransformerMixin):
    
    def __init__(self, cols=None):
        if isinstance(cols, str):
            self.cols = [cols]
        else:
            self.cols = cols
        
        
    def fit(self, X, y):
        # Encode all categorical cols by default
        if self.cols is None:
            self.cols = [col for col in X 
                         if str(X[col].dtype)=='object']

        # Check columns are in X
        for col in self.cols:
            if col not in X:
                raise ValueError('Column \''+col+'\' not in X')

        # Encode each element of each column
        self.maps = dict() #dict to store map for each column
        for col in self.cols:
            tmap = dict()
            uniques = X[col].unique()
            for unique in uniques:
                tmap[unique] = y[X[col]==unique].mean()
            self.maps[col] = tmap
            
        return self

        
    def transform(self, X, y=None):
        Xo = X.copy()
        for col, tmap in self.maps.items():
            vals = np.full(X.shape[0], np.nan)
            for val, mean_target in tmap.items():
                vals[X[col]==val] = mean_target
            Xo[col] = vals
        return Xo
            
            
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X, y)

In [18]:
class TargetEncoderLOO(TargetEncoder):
    
    def __init__(self, cols=None):
        self.cols = cols
        
    def fit(self, X, y):
        # Encode all categorical cols by default
        if self.cols is None:
            self.cols = [col for col in X
                         if str(X[col].dtype)=='object']

        # Check columns are in X
        for col in self.cols:
            if col not in X:
                raise ValueError('Column \''+col+'\' not in X')

        # Encode each element of each column
        self.sum_count = dict()
        for col in self.cols:
            self.sum_count[col] = dict()
            uniques = X[col].unique()
            for unique in uniques:
                ix = X[col]==unique
                self.sum_count[col][unique] = \
                    (y[ix].sum(),ix.sum())
            
        # Return the fit object
        return self

    
    def transform(self, X, y=None):  
        # Create output dataframe
        Xo = X.copy()

        # Use normal target encoding if this is test data
        if y is None:
            for col in self.sum_count:
                vals = np.full(X.shape[0], np.nan)
                for cat, sum_count in self.sum_count[col].items():
                    vals[X[col]==cat] = sum_count[0]/sum_count[1]
                Xo[col] = vals

        # LOO target encode each column
        else:
            for col in self.sum_count:
                vals = np.full(X.shape[0], np.nan)
                for cat, sum_count in self.sum_count[col].items():
                    ix = X[col]==cat
                    vals[ix] = (sum_count[0]-y[ix])/(sum_count[1]-1)
                Xo[col] = vals
            
        # Return encoded DataFrame
        return Xo
      
            
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X, y)

In [19]:
def LOOencoding(X,y):
    y=y.astype(str)
    enc=ce.OneHotEncoder().fit(y)
    y_onehot=enc.transform(y)
    class_names=y_onehot.columns
    
    X_new=pd.DataFrame()
    name=['pm','byc','motor']
    enc_save=[]
    for i in range(len(class_names)):
        cl = class_names[i]
        enc=TargetEncoderLOO()
        enc_save.append(enc)
        enc.fit(X,y_onehot[cl])
        temp=enc.fit_transform(X,y_onehot[cl])
        temp.columns=[str(x)+'_'+str(name[i]) for x in temp.columns]
        X_new=pd.concat([X_new,temp],axis=1)
    return X_new, enc_save

In [20]:
def TargetEncoding(X,y):
    y=y.astype(str)
    enc=ce.OneHotEncoder().fit(y)
    y_onehot=enc.transform(y)
    class_names=y_onehot.columns
    
    X_new=pd.DataFrame()
    name=['pm','byc','motor']
    for i in range(len(class_names)):
        cl = class_names[i]
        enc=TargetEncoder()
        temp=enc.fit_transform(X,y_onehot[cl])
        temp.columns=[str(x)+'_'+str(name[i]) for x in temp.columns]
        X_new=pd.concat([X_new,temp],axis=1)
    return X_new

In [21]:
def make_mean(data,col):
    temp = data.copy()
    y=data[data[col]!=-1].mean()
    
    for index,row in data.iterrows():
        if row[0] == -1:
            temp.loc[index]=y
            
    return temp

# 인코딩 실행

### Dummy & Binary

In [22]:
dt_week_train=dummy_encoding(dt['요일'],['일','월','화','수','목','금','토'],["sun","mon","tue","wed","thu","fri","sat"])
save_df(dt_week_train)

In [23]:
df_slope_train=dummy_encoding(dt['SOILSLOPE'],['0-2%','2-7%','7-15%','15-30%','기타','30-60%','60-100%'],["slope_1","slope_2","slope_3","slope_4","slope_etc","slope_5","slope_6"])
save_df(df_slope_train)

In [24]:
dt_night_train=binary_encoding(dt[['주야']],['야'],['주'],['night'])
save_df(dt_night_train)

In [25]:
dt_at_gender_train=binary_encoding(dt[['성별가해자']],['남'],['여'],['at_gender'])
save_df(dt_at_gender_train)

In [26]:
dt_vt_gender_train=binary_encoding(dt[['성별가해자_1']],['남'],['여'],['vt_gender'])
save_df(dt_vt_gender_train)

In [27]:
dt_road_stat_train=binary_encoding(dt[['노면상태']],['건조'],['젖음/습기','기타','서리/결빙','적설','침수','해빙'],['road_stat'])
save_df(dt_road_stat_train)

In [28]:
dt_alch_train=binary_encoding(dt[['음주측정수치가해자_대분류']],['음주운전','음주상태'],['해당 없음'],['alch'])
save_df(dt_alch_train)

In [29]:
dt_at_prot_train=binary_encoding(dt[['보호장구가해자']],['착용'],['미착용'],['at_prot'])
save_df(dt_at_prot_train)

In [30]:
dt_vt_prot_train=binary_encoding(dt[['보호장구피해자']],['착용','없음'],['미착용','보행자'],['vt_prot'])
save_df(dt_vt_prot_train)

### Ordinal

In [31]:
dt_accident_train= make_ordinal(dt[['사고내용']],[['부상신고'],['경상'],['중상'],['사망']],['accident'],[0.25,0.5,0.75,1])
save_df(dt_accident_train)

In [32]:
dt_at_driving_straight_train=binary_encoding(dt[['행동유형가해자_중분류']],
                                          ['직진관련'],['주정차중','후진 중','주행 중 대기','불명','기타','회전관련'],
                                          ['at_driving_straight'])
save_df(dt_at_driving_straight_train)

In [33]:
dt_at_driving_leftright_train=binary_encoding(dt[['행동유형가해자_중분류']],
                                          ['회전관련'],['주정차중','후진 중','주행 중 대기','불명','기타','직진관련'],
                                          ['at_driving_leftright'])
save_df(dt_at_driving_leftright_train)

In [34]:
dt_road_form_train= make_ordinal(dt[['도로형태_대분류']],[['단일로'],['교차로'],['기타','불명','주차장']]
                                 ,['road_form_B'],[0, 1,-1])
save_df(dt_road_form_train)

In [35]:
save_df(dt[['사고유형_중분류', '법규위반가해자', '행동유형가해자', '도로형태', '교차로형태']])

## binning

In [36]:
def bin_age(x):
    bins= [0,13,16,20,30,40,50,60,70,98,150]
    labels = [0,13,16,20,30,40,50,60,70,-1]
    return pd.cut(x, bins=bins, labels=labels, right=False)

In [37]:
def make_number(x):  # 숫자만 추출
    if str(x) != "미분류":
        numbers = re.sub(r'[^0-9]', '', str(x))
        if numbers:
            return int(numbers)
        else:
            return 0
    else:
        return -1

In [38]:
dt_at_age_train= dt['연령가해자'].apply(make_number)
dt_at_age_train = bin_age(dt_at_age_train).astype('int')
dt_at_age_train_temp=pd.DataFrame(dt_at_age_train)
dt_at_age_train_temp.columns=['at_age']
dt_at_age_train_temp.reset_index(inplace=True,drop=True)
save_df(dt_at_age_train_temp)

In [39]:
dt_vt_age_train= dt['연령피해자'].apply(make_number)
dt_vt_age_train = bin_age(dt_vt_age_train).astype('int')
dt_vt_age_train_temp=pd.DataFrame(dt_vt_age_train)
dt_vt_age_train_temp.columns=['vt_age']
dt_vt_age_train_temp.reset_index(inplace=True,drop=True)

save_df(dt_vt_age_train_temp)

In [40]:
dt_another_train=dt.loc[:,['schoolA','silverA','bycA','road_num','busstop_n','tree_n','subway_n','Class']]
save_df(dt_another_train)

### 남은 전처리 
target 변수 처리, road_form_B 결측치 대체, 100세 이상 삭제, 데이터 셋 나누기(차대차...)

In [41]:
df_try=save_pre.copy()

In [42]:
df_try['group']=dt['사고유형_대분류']

In [43]:
df_try.loc[((df_try['road_form_B']== -1) & (df_try['road_num']>= 2)), 'road_form_B'] = 1
df_try.loc[((df_try['road_form_B']== -1) & (df_try['road_num']< 2)), 'road_form_B'] = 0

In [44]:
df_try = df_try[df_try['at_age'] != -1]
df_try = df_try[df_try['at_gender'] != -1]
df_try = df_try[df_try['vt_age'] != -1]

In [45]:
dt_y=df_try['Class']
dt_X=df_try.drop(['Class'],axis=1,inplace=False)

In [46]:
train_X, test_X, train_y, test_y = train_test_split(dt_X,dt_y,test_size=0.2, random_state=0, stratify = dt_y)

In [47]:
# train_X.reset_index(inplace=True,drop=True)
# test_X.reset_index(inplace=True,drop=True)
# train_y.reset_index(inplace=True,drop=True)
# test_y.reset_index(inplace=True,drop=True)

In [48]:
df_train = pd.concat([train_X,train_y],axis=1)
df_test = pd.concat([test_X,test_y],axis=1)
df_tt = [df_train,df_test]

## dict 만들기

In [49]:
def make_dict(front_df, front_df_name, func_num, func_cond , back_df_name):        
    a_dict={}  
    
    for df1,dfname1 in zip(front_df, front_df_name):   #a는 ordinal 하나당 쪼개진 4개의 데이터프레임 묶음 / 반복 4번
        if(func_num==1):
            temp = make_vvv(df1,func_cond)
        elif(func_num==2):
            temp = make_interaction2(df1,func_cond)
        elif(func_num==3):
            temp = make_drop(df1)
            
        for df2, dfname2 in zip(temp, back_df_name):  # /반복 3번
            a_dict[dfname1+'_' + dfname2]= df2
            
    return a_dict

In [50]:
def make_vvv(df, cond): 
    temp=[]
    
    for i in cond:
        df_temp=df.copy()
        temp.append(df_temp[df_temp['group']==i])
    
    return temp

In [51]:
def make_interaction2(dt_vv2, arr): # arr : ['law_viol','at_activ','road_form','cross_form']
    temp=[]
    df_temp=dt_vv2
    col_driving = np.array(['at_driving_straight','at_driving_leftright',
                            'law_viol',
                           'at_activ', 'night'])
    col_env = np.array(['road_form', 'cross_form', 'schoolA', 'silverA',
                        'bycA', 'road_num', 'busstop_n',
                        'tree_n','subway_n','slope_2', 'slope_3', 'slope_4', 'slope_etc', 'slope_5', 'slope_6'])
    last=['law_viol','at_activ','road_form','cross_form']
    for i in arr:
        df_temp=dt_vv2.copy()
        if i in col_driving:
            for col1 in col_driving:
                if i==col1:
                    col1=i+'_ordinal'
                for col2 in col_env:
                    df_temp[col2 + '*' + col1] = df_temp[col1].mul(df_temp[col2])
        
        elif i in col_env:
            for col1 in col_env:
                if i==col1:
                    col1=i+'_ordinal'
                for col2 in col_driving:

                    df_temp[col2 + '*' + col1] = df_temp[col1].mul(df_temp[col2])
        
        else:
            for col1 in col_driving:
                for col2 in col_env:
                    df_temp[col2 + '*' + col1] = df_temp[col1].mul(df_temp[col2])
                    
                
        if i in last:
            df_temp.drop(i, axis=1, inplace=True)
            df_temp.drop([x + '_ordinal' for x in last if x != i], +axis=1, inplace=True)
        else:
            df_temp.drop([x + '_ordinal' for x in last], axis=1, inplace=True)

        df_temp.drop('acci_M_ordinal', axis=1, inplace=True)
        temp.append(df_temp)  
              
        
#         df_temp.drop(i, axis=1, inplace=True)
#         df_temp.drop('acci_M_ordinal', axis=1, inplace=True)
#         df_temp.drop([x + '_ordinal' for x in arr if x != i], axis=1, inplace=True)
#         temp.append(df_temp)
    
    return temp

In [52]:
def make_drop(df): 
    temp=[]
    
    df_temp=df.copy()
    temp.append(df_temp)
    temp.append(df_temp.drop(['accident','at_age','vt_age','at_gender', 'vt_gender',
        'at_prot', 'vt_prot', 'alch', 
        'mon','tue','wed','thu','fri','sat'],axis=1))
    
    return temp

In [53]:
a_dict_check= make_dict(df_tt, ['df_train','df_test'], 1, ['차대차','차대사람','차량단독'] , ['vv','vp','va'])

In [54]:
dict_key_name=[]
dict_value_df=[]
for key,value in a_dict_check.items():
    dict_key_name.append(key)
    dict_value_df.append(value.drop(['group'],axis=1))

In [55]:
dict_key_name

['df_train_vv',
 'df_train_vp',
 'df_train_va',
 'df_test_vv',
 'df_test_vp',
 'df_test_va']

### 변환 함수
- 결측값 mean 대체 : alch, prot, vt_gender 
- LeaveOneOut encoding
- LOO encoding pm 삭제 후 motor, cycle-> PCA, 
- LOO -> binary

In [56]:
def make_mean2(data):
    temp = data.copy()
    a=data[data==0].count()
    b=data[data==1].count()

    y=b/(a+b)
    for index,row in data.iteritems():
        if row == -1:
            temp.loc[index]=y
            
    return temp

In [57]:
def replace_missing(data):
    data = data[data['at_gender'] != -1]
    data['alch'] = make_mean2(data['alch'])
    data['at_prot'] = make_mean2(data['at_prot'])
    data['vt_prot'] = make_mean2(data['vt_prot'])
    data['vt_gender'] = make_mean2(data['vt_gender'])
    
    looname = ['사고유형_중분류', '법규위반가해자', '행동유형가해자', '도로형태', '교차로형태']

    for j in range(len(looname)):
        loo_h1 = data[looname[j]].value_counts()[data[looname[j]].value_counts() == 1].index
        loo_h1_list = [x for x in loo_h1]

        for i in range(len(loo_h1_list)):
            data.drop(data[data[looname[j]]==loo_h1_list[i]].index, inplace=True)

    return data


In [58]:
def do_loo(data1, data2):
    
    #1. loo encoding
    dt_acci_M,LOOenc_1=LOOencoding(data1[['사고유형_중분류']],data1[['Class']])
    dt_acci_M.columns=['acci_M_motor','acci_M_byc','acci_M_pm']

    dt_acci_M_t =pd.DataFrame()
    for i in LOOenc_1:
        dt_acci_M_test= i.transform(data2[['사고유형_중분류']])
        dt_acci_M_t=pd.concat([dt_acci_M_t,dt_acci_M_test],axis=1)
    dt_acci_M_t.columns=['acci_M_motor','acci_M_byc','acci_M_pm']

    dt_law_viol,LOOenc_2=LOOencoding(data1[['법규위반가해자']],data1[['Class']])
    dt_law_viol.columns=['law_viol_motor','law_viol_byc','law_viol_pm']
#     print(data1.isnull().sum())    

    dt_law_viol_t =pd.DataFrame()
    for i in LOOenc_2:
        dt_law_viol_test= i.transform(data2[['법규위반가해자']])
        dt_law_viol_t=pd.concat([dt_law_viol_t,dt_law_viol_test],axis=1)
    dt_law_viol_t.columns=['law_viol_motor','law_viol_byc','law_viol_pm']

    dt_at_activ, LOOenc_3=LOOencoding(data1[['행동유형가해자']],data1[['Class']])
    dt_at_activ.columns=['at_activ_motor','at_activ_byc','at_activ_pm']
    
    dt_at_activ_t =pd.DataFrame()
    for i in LOOenc_3:
        dt_at_activ_test= i.transform(data2[['행동유형가해자']])
        dt_at_activ_t=pd.concat([dt_at_activ_t,dt_at_activ_test],axis=1)
    dt_at_activ_t.columns=['at_activ_motor','at_activ_byc','at_activ_pm']

    dt_road_form, LOOenc_4=LOOencoding(data1[['도로형태']],data1[['Class']])
    dt_road_form.columns=['road_form_motor','road_form_byc','road_form_pm']

    dt_road_form_t =pd.DataFrame()
    for i in LOOenc_4:
        dt_road_form_test= i.transform(data2[['도로형태']])
        dt_road_form_t=pd.concat([dt_road_form_t,dt_road_form_test],axis=1)
    dt_road_form_t.columns=['road_form_motor','road_form_byc','road_form_pm']

    dt_cross_form, LOOenc_7=LOOencoding(data1[['교차로형태']],data1[['Class']])
    dt_cross_form.columns=['cross_form_motor','cross_form_byc','cross_form_pm']

    dt_cross_form_t =pd.DataFrame()
    for i in LOOenc_7:
        dt_cross_form_test= i.transform(data2[['교차로형태']])
        dt_cross_form_t=pd.concat([dt_cross_form_t,dt_cross_form_test],axis=1)
    dt_cross_form_t.columns=['cross_form_motor','cross_form_byc','cross_form_pm']
#     print(dt_at_activ.isnull().sum())
        
    train = pd.concat([dt_acci_M,dt_law_viol,dt_at_activ,dt_road_form,dt_cross_form],axis=1)
    test = pd.concat([dt_acci_M_t,dt_law_viol_t,dt_at_activ_t,dt_road_form_t,dt_cross_form_t],axis=1)
    
    #2. stnd scaling
    col = ['acci_M_pm', 'law_viol_pm', 'at_activ_pm', 'road_form_pm', 'cross_form_pm']
    col2 = ['acci_M', 'law_viol', 'at_activ', 'road_form', 'cross_form']

    train.drop(col, axis=1, inplace=True)
    test.drop(col, axis=1, inplace=True)

    standardscaler = StandardScaler()
    standardscaler.fit(train)
    stnd_train_loo = pd.DataFrame(standardscaler.transform(train), columns=train.columns)
    stnd_test_loo = pd.DataFrame(standardscaler.transform(test), columns=test.columns)


    #3. pca
    dt1 = pd.DataFrame()
    dt2 = pd.DataFrame()
    
    for i in range(5):
        pca = PCA()
        principalComponents1 = pca.fit_transform(stnd_train_loo.iloc[:,(i*2):(2+i*2)])
        dt1[col2[i]] = pd.DataFrame(principalComponents1).iloc[:,0]
        principalComponents2 = pca.transform(stnd_test_loo.iloc[:,(i*2):(2+i*2)])
        dt2[col2[i]] = pd.DataFrame(principalComponents2).iloc[:,0]

    data1 = pd.concat([data1.reset_index(drop=True),dt1], axis=1)
    data2 = pd.concat([data2.reset_index(drop=True),dt2], axis=1)

    data1.drop(list(data1.filter(regex = '._motor')), axis = 1, inplace=True)
    data1.drop(list(data1.filter(regex = '._byc')), axis = 1, inplace=True)

    data2.drop(list(data2.filter(regex = '._motor')), axis = 1, inplace=True)
    data2.drop(list(data2.filter(regex = '._byc')), axis = 1, inplace=True)

    return data1, data2

In [59]:
def loo_to_ordinal(data, arr):
    def loo_to_binary(data, i, item, new):
        loo= ['사고유형_중분류', '법규위반가해자', '행동유형가해자', '도로형태', '교차로형태']
        newcol = binary_encoding(data[[loo[i]]], 
               item, [x for x in data[loo[i]].unique() if x not in item],
               [new])
        return newcol

    def loo_to_ordinal(data, i, item, new):
        loo= ['사고유형_중분류', '법규위반가해자', '행동유형가해자', '도로형태', '교차로형태']
        newcol = make_ordinal(data[[loo[i]]], 
                   item, [new], [0,0.5,1])
        return newcol
    
    data = pd.concat([data,(loo_to_binary(data, 0, arr[0], 'acci_M_ordinal'))],axis=1)
    data = pd.concat([data,(loo_to_ordinal(data, 1, arr[1], 'law_viol_ordinal'))],axis=1)
    data = pd.concat([data,(loo_to_ordinal(data, 2, arr[2], 'at_activ_ordinal'))],axis=1)
    data = pd.concat([data,(loo_to_binary(data, 3, arr[3], 'road_form_ordinal'))],axis=1)
    data = pd.concat([data,(loo_to_ordinal(data, 4, arr[4], 'cross_form_ordinal'))],axis=1)
    
    return data

In [60]:
array1 = [['추돌', '후진중충돌'],
          
          [['기타', '중앙선 침범', '안전운전 의무 불이행'],
           ['교차로 통행방법 위반'], ['안전거리 미확보', '신호위반', '과속' ]]
          , 
[['주행 중 대기', '주정차중', 'U턴 중', '진로변경 중', '후진 중' ],
                                        ['우회전 중', '좌회전 중', '기타'],
                                        ['좌우회전중','직진 중', '앞지르기 중', '불명']],
['기타', '주차장', '불명'],
          
[['교차로 - 사지', '교차로 - 오지이상', '교차로 - 회전'],
                                        ['교차로 - 삼지'],
                                        ['교차로아님']]]

In [61]:
array2 = [['보도통행중', '기타'],
          
          [['기타', '안전거리 미확보', '안전운전 의무 불이행'],
           ['보행자 보호의무 위반'], ['교차로 통행방법 위반','중앙선 침범', '신호위반', '과속' ]]
          , 
[['주정차중', 'U턴 중', '진로변경 중', '후진 중' ],
                                        ['우회전 중', '좌회전 중', '기타'],
                                        ['좌우회전중','직진 중', '앞지르기 중', '불명']],
['교량위','기타', '지하차도(도로)내', '주차장'],
          
[['교차로 - 사지', '교차로 - 오지이상', '교차로 - 회전'],
                                        ['교차로 - 삼지'],
                                        ['교차로아님']]]

In [62]:
array3 = [['기타', '주/정차차량 충돌', '도로이탈'],
          
          [['기타', '안전운전 의무 불이행'],
           ['교차로 통행방법 위반'], [ '중앙선 침범','안전거리 미확보', '신호위반', '과속' ]]
          , 
[['주정차중', 'U턴 중', '진로변경 중', '후진 중' ],
                                        ['우회전 중', '좌회전 중', '기타'],
                                        ['좌우회전중','직진 중', '앞지르기 중', '불명']],
['교량위', '터널안', '기타', '지하차도(도로)내', '주차장'],
          
[['교차로 - 사지', '교차로 - 오지이상', '교차로 - 회전'],
                                        ['교차로 - 삼지'],
                                        ['교차로아님']]]

In [63]:
array = [array1, array2, array3]

In [65]:
def transform(data1, data2, arr):
    data1 = replace_missing(data1)
    data2 = replace_missing(data2)
    
    data1, data2 = do_loo(data1, data2)
    

    data1 = loo_to_ordinal(data1, arr)
    data2 = loo_to_ordinal(data2, arr)
    return data1, data2

In [ ]:
tt_num=int(len(dict_value_df)/2)
for i in range(0,tt_num):
    dict_value_df[i],dict_value_df[i+tt_num]=transform(dict_value_df[i],dict_value_df[i+tt_num],array[i])

In [67]:
b_dict_check= make_dict(dict_value_df, dict_key_name, 2, ['a','law_viol','at_activ','road_form','cross_form'] , ['original','law_viol_ordinal','at_activ_ordinal','road_form_ordinal','cross_form_ordinal'])

In [68]:
dict_key_name=[]
dict_value_df=[]
for key,value in b_dict_check.items():
    dict_key_name.append(key)
    dict_value_df.append(value)

In [69]:
c_dict_check= make_dict(dict_value_df, dict_key_name, 3,0, ['all','drop'])

In [70]:
dict_key_name=[]
dict_value_df=[]
for key,value in c_dict_check.items():
    dict_key_name.append(key)
    dict_value_df.append(value)

In [71]:
for i in range(len(dict_key_name)):
    dict_value_df[i].drop(['사고유형_중분류', '법규위반가해자', '행동유형가해자', '도로형태', '교차로형태'],axis=1,inplace=True)

In [72]:
def minmaxfunction(df_train,df_test,check):
    minMaxScaler = MinMaxScaler()
    df_train.reset_index(drop=True, inplace=True)
    df_test.reset_index(drop=True, inplace=True)  
    df_train_tmp = df_train.drop('Class',axis=1)
    df_test_tmp=df_test.drop('Class',axis=1)

    minMaxScaler.fit(df_train_tmp)
    df_train_loo_minmax =pd.DataFrame(minMaxScaler.transform(df_train_tmp))
    df_test_loo_minmax =pd.DataFrame(minMaxScaler.transform(df_test_tmp))
    
    df_train_loo_minmax['Class']=df_train['Class']
    colname = [x for x in df_train.columns.drop('Class')]
    colname.append('Class')
    
    df_test_loo_minmax['class']=df_test['Class']
    colname = [x for x in df_test.columns.drop('Class')]
    colname.append('Class')
    
    df_train_loo_minmax.columns = colname
    df_test_loo_minmax.columns = colname
    if check%2:
        df_train_loo_minmax['at_age']=df_train['at_age']
        df_train_loo_minmax['vt_age']=df_train['vt_age']
        df_test_loo_minmax['at_age']=df_test['at_age']
        df_test_loo_minmax['vt_age']=df_test['vt_age']
    
    return df_train_loo_minmax, df_test_loo_minmax

In [73]:
dict_value_df2=dict_value_df.copy()

In [ ]:
tt_num=int(len(dict_value_df)/2)

for i in range(0,tt_num):
    dict_value_df2[i],dict_value_df2[i+tt_num]=minmaxfunction(dict_value_df[i],dict_value_df[i+tt_num],i+1)

In [75]:
for num, i in zip(range(len(dict_value_df2)),dict_value_df2):
    dict_value_df2[num]=i.fillna(0)

In [1]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from pycaret.classification import *
from sklearn.metrics import recall_score
from sklearn import metrics

In [114]:
def pycaret_all(dflist,dfname):
    dict_coef={}
    tt_num=1   
#     tt_num=int(len(dflist)/2)
    accu_score=[]
    rec_score=[]
    cfs_matrix=[]
    cls_report=[]
    
    for i in range(0,tt_num):
        i=21
        if i % 2 == 0:
            clf = setup(data =dflist[i], target = 'Class', fix_imbalance=True, 
                        numeric_features=['road_num','busstop_n','tree_n','subway_n'], 
                        categorical_features=['at_age','vt_age'], session_id=0,feature_interaction=False,
                        feature_selection=True, normalize=False, silent = True)
        else: 
            clf = setup(data =dflist[i], target = 'Class', fix_imbalance=True, 
                        numeric_features=['road_num','busstop_n','tree_n','subway_n'], 
                        session_id=0, feature_interaction=False,
                        feature_selection=True, normalize=False, silent = True)
            
        lr = create_model('lr')
#         tuned_lr = tune_model(lr)
        final_model = finalize_model(lr)
        
        test_df = dflist[30+i].copy()
        label_test_y = test_df["Class"]
        test_df.drop(['Class'], axis = 1, inplace = True)
        predictions = predict_model(final_model, data = test_df)

        pred_label=predictions['Label']
        pred_score=predictions['Score']

        #print(accuracy_score(label_test_y , pred_label))
        #print(recall_score(label_test_y , pred_label, average=None))
        print('done!')
        
        accu_score.append(accuracy_score(label_test_y , pred_label))
        rec_score.append(recall_score(label_test_y , pred_label, average=None))
        cfs_matrix.append(metrics.confusion_matrix(label_test_y, pred_label, labels=["byc", "pm", "motor"]))
        cls_report.append(metrics.classification_report(label_test_y, pred_label, labels=["byc", "pm", "motor"]))

        fi0 = pd.DataFrame({'Feature': get_config('X_train').columns, 'Value' : final_model.coef_[0]})
        fi_2 = pd.concat([fi0,pd.DataFrame(final_model.coef_[1]),pd.DataFrame(final_model.coef_[2])],axis=1).sort_values(by='Value', key=abs,ascending=False)
        fi_2.columns = ['feature','byc','pm','motor']
        fi_2['sum'] = np.abs(fi_2['byc']) + np.abs(fi_2['pm']) + np.abs(fi_2['motor'])
        fi_2 = fi_2.sort_values(by = 'sum', ascending=False)
        dict_coef[dfname[i]+'_coef']=fi_2
        
        
        return accu_score,rec_score,dict_coef

In [115]:
accuracy_check,recall_check, coef_check=pycaret_all(dict_value_df2,dict_key_name)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.5424,0.6181,0.5700,0.7559,0.6033,0.1987,0.2368
1,0.6441,0.7657,0.6913,0.7791,0.6849,0.2952,0.3265
2,0.6102,0.6165,0.3652,0.7195,0.6554,0.1561,0.1652
3,0.5517,0.6606,0.5217,0.7514,0.6188,0.1696,0.2001
4,0.5517,0.6589,0.3362,0.7514,0.6321,0.1353,0.1560
5,0.6207,0.7287,0.3652,0.8039,0.6972,0.2295,0.2564
6,0.5690,0.7491,0.6623,0.7680,0.6296,0.2015,0.2377
7,0.5690,0.7292,0.4179,0.7528,0.6338,0.1794,0.2066
8,0.5172,0.6821,0.3961,0.7764,0.5908,0.1655,0.2052
9,0.5345,0.7096,0.4370,0.6985,0.5913,0.1182,0.1330


done!


In [116]:
accuracy_check

[0.5147058823529411]

In [117]:
recall_check

[array([0.2173913 , 0.33333333, 0.56      ])]

In [118]:
dict_key_name[21]

'df_train_va_original_drop'

In [119]:
coef_check['df_train_va_original_drop_coef'].head(40)

,feature,byc,pm,motor,sum
91,acci_M,0.2611,2.3859,-2.6470,5.2940
12,road_num*night,0.8342,-2.1778,1.3436,4.3556
92,road_form,1.9744,-0.2845,-1.6899,3.9487
69,at_driving_straight_0.0,0.3410,-1.7820,1.4410,3.5640
67,road_form*at_driving_leftright,0.9136,-1.7506,0.8370,3.5011
27,at_activ,0.4586,1.2799,-1.7385,3.4771
9,tree_n*night,1.3268,-1.7352,0.4084,3.4705
85,slope_etc*law_viol,1.2343,0.4600,-1.6943,3.3885
86,road_form*night,-1.6732,0.2677,1.4055,3.3464
78,slope_etc_1.0,1.6676,-0.6617,-1.0058,3.3351


In [ ]:
# %save modeling_with_fin_at_dataset.py 1-300